In [1]:
"""
Solving small linear systems via SVD.
case 0. Invertible matrix (demonstrates that SVD
        is equivalento matrix inversion in the
        simplest case)
case 1. Non square matrix (over or under determined 
        system). Here matrix inversion does not work.
        SVD finds the best approximation.
"""
import torch

In [2]:
%run 4.3.2-common.ipynb

In [3]:
# Case 0: Invertible square matrix
A = torch.tensor([[1, 2, 1], [2, 2, 3], [1, 3, 3]], dtype=torch.float)
b = torch.tensor([8, 15, 16], dtype=torch.float)

# Solve this via matrix inversion
# i.e x = inv(A) *  b
x_0 = torch.matmul(torch.linalg.inv(A), b)
print("Solution via inverse: {}".format(x_0))

# Computation of inverse can be unstable. 
# Now solve this via SVD instead.
U, S, V_t = torch.linalg.svd(A)

# A*x = b ==> U*S*V_t*x = b
#         ==> S * V_t*x = U_t*b 
# (inv(U) is U_t, U being orthogonal)
S_V_t_x = torch.matmul(U.T, b)

# V_t*x = inv(S) * U_t * b  
# Inv(S) is easily computable as S is diagonal
S_inv = torch.diag(1 / S) 
V_t_x = torch.matmul(S_inv, S_V_t_x)

# x= V * inv(S) * U_t * b 
x_1 = torch.matmul(V_t.T, V_t_x)
print("Solution via SVD: {}".format(x_1))

# assert that the two solutions are more
# or less identical
assert torch.allclose(x_0, x_1)

Solution via inverse: tensor([1.0000, 2.0000, 3.0000])
Solution via SVD: tensor([1.0000, 2.0000, 3.0000])


In [4]:
# Case 1: Non square matrices
# Let us revisit our cat brain dataset 

A = torch.tensor([[0.11, 0.09], [0.01, 0.02],
              [0.98, 0.91], [0.12, 0.21],
              [0.98, 0.99], [0.85, 0.87],
              [0.03, 0.14], [0.55, 0.45],
              [0.49, 0.51], [0.99, 0.01],
              [0.02, 0.89], [0.31, 0.47],
              [0.55, 0.29], [0.87, 0.76],
              [0.63, 0.24]])
A = torch.column_stack((A, torch.ones(15))) 
b = torch.tensor([-0.8, -0.97, 0.89, -0.67,
              0.97, 0.72, -0.83, 0.00,
              0.00, 0.00, -0.09, -0.22,
              -0.16, 0.63, 0.37])

# One way to solve for this is via pseudo-inverse
# i.e x = pseudo_inv(A) *  b
x_0 = torch.matmul(torch.linalg.pinv(A), b)
print("Solution via pseudo-inverse:\n{}".\
      format(x_0))

# Computation of inverse can be unstable. 
# So let us try solving this via SVD instead

# Now solve this via SVD instead.
U, S, V_t = torch.linalg.svd(A,
                          full_matrices=False)
# A*x = b ==> U*S*V_t*x = b
#         ==> S * V_t*x = U_t*b 
# As inv(U) is U_t
S_V_t_x = torch.matmul(U.T, b)

# V_t*x = inv(S) * U_t * b  
# Inv(S) is easily computable as it is diagonal
S_inv = torch.diag(1 / S) 
V_t_x = torch.matmul(S_inv, S_V_t_x)

# x= V * inv(S) * U_t * b 
x_1 = torch.matmul(V_t.T, V_t_x)
print("Solution via SVD:\n{}".format(x_1))

assert torch.allclose(x_0, x_1)

Solution via pseudo-inverse:
tensor([ 1.0766,  0.8976, -0.9582])
Solution via SVD:
tensor([ 1.0766,  0.8976, -0.9582])
